In [1]:
import pandas as pd
import math
import collections


In [2]:
teams_data = pd.read_csv("kaggle_data/Teams.csv")
teamList = teams_data['Team_Name'].tolist()
reg_season_detail = pd.read_csv("kaggle_data/RegularSeasonDetailedResults.csv")
reg_season_compact_pd = pd.read_csv('kaggle_data/RegularSeasonCompactResults.csv')
seasons_data = pd.read_csv("kaggle_data/Seasons.csv")
tourney_compact = pd.read_csv("kaggle_data/TourneyCompactResults.csv")
tourney_detail = pd.read_csv("kaggle_data/TourneyDetailedResults.csv")
tourney_seeds_pd = pd.read_csv("kaggle_data/TourneySeeds.csv")
tourney_slots = pd.read_csv("kaggle_data/TourneySlots.csv")
conference_pd = pd.read_csv('Conference.csv')
tourney_results_pd = pd.read_csv('TourneyResults.csv')
NCAAChampionsList = tourney_results_pd['NCAA Champion'].tolist()
SampleSubmission = pd.read_csv("kaggle_data/SampleSubmission.csv")

## Get Season Data
Credit to https://github.com/adeshpande3/March-Madness-2017 for the aggregation helper functions, and logic

In [3]:
listACCteams = ['North Carolina','Virginia','Florida St','Louisville','Notre Dame','Syracuse','Duke','Virginia Tech','Georgia Tech','Miami','Wake Forest','Clemson','NC State','Boston College','Pittsburgh']
listPac12teams = ['Arizona','Oregon','UCLA','California','USC','Utah','Washington St','Stanford','Arizona St','Colorado','Washington','Oregon St']
listSECteams = ['Kentucky','South Carolina','Florida','Arkansas','Alabama','Tennessee','Mississippi St','Georgia','Ole Miss','Vanderbilt','Auburn','Texas A&M','LSU','Missouri']
listBig10teams = ['Maryland','Wisconsin','Purdue','Northwestern','Michigan St','Indiana','Iowa','Michigan','Penn St','Nebraska','Minnesota','Illinois','Ohio St','Rutgers']
listBig12teams = ['Kansas','Baylor','West Virginia','Iowa St','TCU','Kansas St','Texas Tech','Oklahoma St','Texas','Oklahoma']
listBigEastteams = ['Butler','Creighton','DePaul','Georgetown','Marquette','Providence','Seton Hall','St John\'s','Villanova','Xavier']

def checkPower6Conference(team_id):
    teamName = teams_data.values[team_id-1101][1]
    if (teamName in listACCteams or teamName in listBig10teams or teamName in listBig12teams
       or teamName in listSECteams or teamName in listPac12teams or teamName in listBigEastteams):
        return 1
    else:
        return 0

def handleDifferentCSV(df):
    # The stats CSV is a lit different in terms of naming so below is just some data cleaning
    df['School'] = df['School'].replace('(State)', 'St', regex=True) 
    df['School'] = df['School'].replace('Albany (NY)', 'Albany NY') 
    df['School'] = df['School'].replace('Boston University', 'Boston Univ')
    df['School'] = df['School'].replace('Central Michigan', 'C Michigan')
    df['School'] = df['School'].replace('(Eastern)', 'E', regex=True)
    df['School'] = df['School'].replace('Louisiana St', 'LSU')
    df['School'] = df['School'].replace('North Carolina St', 'NC State')
    df['School'] = df['School'].replace('Southern California', 'USC')
    df['School'] = df['School'].replace('University of California', 'California', regex=True) 
    df['School'] = df['School'].replace('American', 'American Univ')
    df['School'] = df['School'].replace('Arkansas-Little Rock', 'Ark Little Rock')
    df['School'] = df['School'].replace('Arkansas-Pine Bluff', 'Ark Pine Bluff')
    df['School'] = df['School'].replace('Bowling Green St', 'Bowling Green')
    df['School'] = df['School'].replace('Brigham Young', 'BYU')
    df['School'] = df['School'].replace('Cal Poly', 'Cal Poly SLO')
    df['School'] = df['School'].replace('Centenary (LA)', 'Centenary')
    df['School'] = df['School'].replace('Central Connecticut St', 'Central Conn')
    df['School'] = df['School'].replace('Charleston Southern', 'Charleston So')
    df['School'] = df['School'].replace('Coastal Carolina', 'Coastal Car')
    df['School'] = df['School'].replace('College of Charleston', 'Col Charleston')
    df['School'] = df['School'].replace('Cal St Fullerton', 'CS Fullerton')
    df['School'] = df['School'].replace('Cal St Sacramento', 'CS Sacramento')
    df['School'] = df['School'].replace('Cal St Bakersfield', 'CS Bakersfield')
    df['School'] = df['School'].replace('Cal St Northridge', 'CS Northridge')
    df['School'] = df['School'].replace('East Tennessee St', 'ETSU')
    df['School'] = df['School'].replace('Detroit Mercy', 'Detroit')
    df['School'] = df['School'].replace('Fairleigh Dickinson', 'F Dickinson')
    df['School'] = df['School'].replace('Florida Atlantic', 'FL Atlantic')
    df['School'] = df['School'].replace('Florida Gulf Coast', 'FL Gulf Coast')
    df['School'] = df['School'].replace('Florida International', 'Florida Intl')
    df['School'] = df['School'].replace('George Washington', 'G Washington')
    df['School'] = df['School'].replace('Georgia Southern', 'Ga Southern')
    df['School'] = df['School'].replace('Gardner-Webb', 'Gardner Webb')
    df['School'] = df['School'].replace('Illinois-Chicago', 'IL Chicago')
    df['School'] = df['School'].replace('Kent St', 'Kent')
    df['School'] = df['School'].replace('Long Island University', 'Long Island')
    df['School'] = df['School'].replace('Loyola Marymount', 'Loy Marymount')
    df['School'] = df['School'].replace('Loyola (MD)', 'Loyola MD')
    df['School'] = df['School'].replace('Loyola (IL)', 'Loyola-Chicago')
    df['School'] = df['School'].replace('Massachusetts', 'MA Lowell')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    df['School'] = df['School'].replace('Miami (FL)', 'Miami FL')
    df['School'] = df['School'].replace('Miami (OH)', 'Miami OH')
    df['School'] = df['School'].replace('Missouri-Kansas City', 'Missouri KC')
    df['School'] = df['School'].replace('Monmouth', 'Monmouth NJ')
    df['School'] = df['School'].replace('Mississippi Valley St', 'MS Valley St')
    df['School'] = df['School'].replace('Montana St', 'MTSU')
    df['School'] = df['School'].replace('Northern Colorado', 'N Colorado')
    df['School'] = df['School'].replace('North Dakota St', 'N Dakota St')
    df['School'] = df['School'].replace('Northern Illinois', 'N Illinois')
    df['School'] = df['School'].replace('Northern Kentucky', 'N Kentucky')
    df['School'] = df['School'].replace('North Carolina A&T', 'NC A&T')
    df['School'] = df['School'].replace('North Carolina Central', 'NC Central')
    df['School'] = df['School'].replace('Pennsylvania', 'Penn')
    df['School'] = df['School'].replace('South Carolina St', 'S Carolina St')
    df['School'] = df['School'].replace('Southern Illinois', 'S Illinois')
    df['School'] = df['School'].replace('UC-Santa Barbara', 'Santa Barbara')
    df['School'] = df['School'].replace('Southeastern Louisiana', 'SE Louisiana')
    df['School'] = df['School'].replace('Southeast Missouri St', 'SE Missouri St')
    df['School'] = df['School'].replace('Stephen F. Austin', 'SF Austin')
    df['School'] = df['School'].replace('Southern Methodist', 'SMU')
    df['School'] = df['School'].replace('Southern Mississippi', 'Southern Miss')
    df['School'] = df['School'].replace('Southern', 'Southern Univ')
    df['School'] = df['School'].replace('St. Bonaventure', 'St Bonaventure')
    df['School'] = df['School'].replace('St. Francis (NY)', 'St Francis NY')
    df['School'] = df['School'].replace('Saint Francis (PA)', 'St Francis PA')
    df['School'] = df['School'].replace('St. John\'s (NY)', 'St John\'s')
    df['School'] = df['School'].replace('Saint Joseph\'s', 'St Joseph\'s PA')
    df['School'] = df['School'].replace('Saint Louis', 'St Louis')
    df['School'] = df['School'].replace('Saint Mary\'s (CA)', 'St Mary\'s CA')
    df['School'] = df['School'].replace('Mount Saint Mary\'s', 'Mt St Mary\'s')
    df['School'] = df['School'].replace('Saint Peter\'s', 'St Peter\'s')
    df['School'] = df['School'].replace('Texas A&M-Corpus Christian', 'TAM C. Christian')
    df['School'] = df['School'].replace('Texas Christian', 'TCU')
    df['School'] = df['School'].replace('Tennessee-Martin', 'TN Martin')
    df['School'] = df['School'].replace('Texas-Rio Grande Valley', 'UTRGV')
    df['School'] = df['School'].replace('Texas Southern', 'TX Southern')
    df['School'] = df['School'].replace('Alabama-Birmingham', 'UAB')
    df['School'] = df['School'].replace('UC-Davis', 'UC Davis')
    df['School'] = df['School'].replace('UC-Irvine', 'UC Irvine')
    df['School'] = df['School'].replace('UC-Riverside', 'UC Riverside')
    df['School'] = df['School'].replace('Central Florida', 'UCF')
    df['School'] = df['School'].replace('Louisiana-Lafayette', 'ULL')
    df['School'] = df['School'].replace('Louisiana-Monroe', 'ULM')
    df['School'] = df['School'].replace('Maryland-Baltimore County', 'UMBC')
    df['School'] = df['School'].replace('North Carolina-Asheville', 'UNC Asheville')
    df['School'] = df['School'].replace('North Carolina-Greensboro', 'UNC Greensboro')
    df['School'] = df['School'].replace('North Carolina-Wilmington', 'UNC Wilmington')
    df['School'] = df['School'].replace('Nevada-Las Vegas', 'UNLV')
    df['School'] = df['School'].replace('Texas-Arlington', 'UT Arlington')
    df['School'] = df['School'].replace('Texas-San Antonio', 'UT San Antonio')
    df['School'] = df['School'].replace('Texas-El Paso', 'UTEP')
    df['School'] = df['School'].replace('Virginia Commonwealth', 'VA Commonwealth')
    df['School'] = df['School'].replace('Western Carolina', 'W Carolina')
    df['School'] = df['School'].replace('Western Illinois', 'W Illinois')
    df['School'] = df['School'].replace('Western Kentucky', 'WKU')
    df['School'] = df['School'].replace('Western Michigan', 'W Michigan')
    df['School'] = df['School'].replace('Abilene Christian', 'Abilene Chr')
    df['School'] = df['School'].replace('Montana State', 'Montana St')
    df['School'] = df['School'].replace('Central Arkansas', 'Cent Arkansas')
    df['School'] = df['School'].replace('Houston Baptist', 'Houston Bap')
    df['School'] = df['School'].replace('South Dakota St', 'S Dakota St')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    return df

def getTeamID(name):
    return teams_data[teams_data['Team_Name'] == name].values[0][0]

def getTeamName(team_id):
    return teams_data[teams_data['Team_Id'] == team_id].values[0][1]

def getNumChampionships(team_id):
    name = getTeamName(team_id)
    return NCAAChampionsList.count(name)

def getListForURL(team_list):
    team_list = [x.lower() for x in team_list]
    team_list = [t.replace(' ', '-') for t in team_list]
    team_list = [t.replace('st', 'state') for t in team_list]
    team_list = [t.replace('northern-dakota', 'north-dakota') for t in team_list]
    team_list = [t.replace('nc-', 'north-carolina-') for t in team_list]
    team_list = [t.replace('fl-', 'florida-') for t in team_list]
    team_list = [t.replace('ga-', 'georgia-') for t in team_list]
    team_list = [t.replace('lsu', 'louisiana-state') for t in team_list]
    team_list = [t.replace('maristate', 'marist') for t in team_list]
    team_list = [t.replace('stateate', 'state') for t in team_list]
    team_list = [t.replace('northernorthern', 'northern') for t in team_list]
    team_list = [t.replace('usc', 'southern-california') for t in team_list]
    base = 'http://www.sports-reference.com/cbb/schools/'
    for team in team_list:
        url = base + team + '/'
getListForURL(teamList);

# Function for handling the annoying cases of Florida and FL, as well as State and St
def handleCases(arr):
    indices = []
    listLen = len(arr)
    for i in range(listLen):
        if (arr[i] == 'St' or arr[i] == 'FL'):
            indices.append(i)
    for p in indices:
        arr[p-1] = arr[p-1] + ' ' + arr[p]
    for i in range(len(indices)): 
        arr.remove(arr[indices[i] - i])
    return arr

def checkConferenceChamp(team_id, year):
    year_conf_pd = conference_pd[conference_pd['Year'] == year]
    champs = year_conf_pd['Regular Season Champ'].tolist()
    # For handling cases where there is more than one champion
    champs_separated = [words for segments in champs for words in segments.split()]
    name = getTeamName(team_id)
    champs_separated = handleCases(champs_separated)
    if (name in champs_separated):
        return 1
    else:
        return 0
    
def checkConferenceTourneyChamp(team_id, year):
    year_conf_pd = conference_pd[conference_pd['Year'] == year]
    champs = year_conf_pd['Tournament Champ'].tolist()
    name = getTeamName(team_id)
    if (name in champs):
        return 1
    else:
        return 0
    
def getTourneyAppearances(team_id):
    return len(tourney_seeds_pd[tourney_seeds_pd['Team'] == team_id].index)

In [4]:
def getSeasonData(team_id, year):
    # The data frame below holds stats for every single game in the given year
    year_data_pd = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
    # Finding number of points per game
    gamesWon = year_data_pd[year_data_pd.Wteam == team_id] 
    totalPointsScored = gamesWon['Wscore'].sum()
    gamesLost = year_data_pd[year_data_pd.Lteam == team_id] 
    totalGames = gamesWon.append(gamesLost)
    numGames = len(totalGames.index)
    totalPointsScored += gamesLost['Lscore'].sum()
    
    # Finding number of points per game allowed
    totalPointsAllowed = gamesWon['Lscore'].sum()
    totalPointsAllowed += gamesLost['Wscore'].sum()
    
    stats_SOS_pd = pd.read_csv('MMStats/MMStats_'+str(year)+'.csv')
    stats_SOS_pd = handleDifferentCSV(stats_SOS_pd)
    ratings_pd = pd.read_csv('RatingStats/RatingStats_'+str(year)+'.csv')
    ratings_pd = handleDifferentCSV(ratings_pd)
    
    name = getTeamName(team_id)
    team = stats_SOS_pd[stats_SOS_pd['School'] == name]
    team_rating = ratings_pd[ratings_pd['School'] == name]
    if (len(team.index) == 0 or len(team_rating.index) == 0): #Can't find the team
        total3sMade = 0
        totalTurnovers = 0
        totalAssists = 0
        sos = 0
        totalRebounds = 0
        srs = 0
        totalSteals = 0
    else:
        total3sMade = team['X3P'].values[0]
        totalTurnovers = team['TOV'].values[0]
        if (math.isnan(totalTurnovers)):
            totalTurnovers = 0
        totalAssists = team['AST'].values[0]
        if (math.isnan(totalAssists)):
            totalAssists = 0
        sos = team['SOS'].values[0]
        srs = team['SRS'].values[0]
        totalRebounds = team['TRB'].values[0]
        if (math.isnan(totalRebounds)):
            totalRebounds = 0
        totalSteals = team['STL'].values[0]
        if (math.isnan(totalSteals)):
            totalSteals = 0
    
    #Finding tournament seed for that year
    tourneyYear = tourney_seeds_pd[tourney_seeds_pd['Season'] == year]
    seed = tourneyYear[tourneyYear['Team'] == team_id]
    if (len(seed.index) != 0):
        seed = seed.values[0][1]
        tournamentSeed = int(seed[1:3])
    else:
        tournamentSeed = 25 #Not sure how to represent if a team didn't make the tourney
    
    # Finding number of wins and losses
    numWins = len(gamesWon.index)
    # There are some teams who may have dropped to Division 2, so they won't have games 
    # a certain year. In this case, we don't want to divide by 0, so we'll just set the
    # averages to 0 instead
    if numGames == 0:
        avgPointsScored = 0
        avgPointsAllowed = 0
        avg3sMade = 0
        avgTurnovers = 0
        avgAssists = 0
        avgRebounds = 0
        avgSteals = 0
    else:
        avgPointsScored = totalPointsScored/numGames
        avgPointsAllowed = totalPointsAllowed/numGames
        avg3sMade = total3sMade/numGames
        avgTurnovers = totalTurnovers/numGames
        avgAssists = totalAssists/numGames
        avgRebounds = totalRebounds/numGames
        avgSteals = totalSteals/numGames
    #return [numWins, sos, srs]
    #return [numWins, avgPointsScored, avgPointsAllowed, checkPower6Conference(team_id), avg3sMade, avg3sAllowed, avgTurnovers,
    #        tournamentSeed, getStrengthOfSchedule(team_id, year), getTourneyAppearances(team_id)]
    return [numWins, avgPointsScored, avgPointsAllowed, checkPower6Conference(team_id), avg3sMade, avgAssists, avgTurnovers,
           checkConferenceChamp(team_id, year), checkConferenceTourneyChamp(team_id, year), tournamentSeed,
            sos, srs, avgRebounds, avgSteals, getTourneyAppearances(team_id), getNumChampionships(team_id)]

### End of https://github.com/adeshpande3/March-Madness-2017 stuff

In [5]:
### This is adashpande3's as well, but heavily modified
def createSeasonDict(year):
    seasonDictionary = collections.defaultdict(list)
    for team in teamList:
        team_id = teams_data[teams_data['Team_Name'] == team].values[0][0]
        team_vector = getSeasonData(team_id, year)
        
        vector_headers = ["num_wins", "avg_points_scored", "avg_points_allowed", "power6", "avg3s_made", "avg_assists",
                         "avg_turnovers", "conference_chap_this_year", "conference_tourney_champ_this_year", "tournament_seed",
                         "sos", "srs", "avg_rebounds", "avg_steals", "num_tourney_appearances", "num_championships"]
        seasonDictionary[team_id] = dict(zip(vector_headers, team_vector))
        
        del seasonDictionary[team_id]["power6"] #we dont' think this is useful
        
        #seasonDictionary[team_id] = team_vector
    return seasonDictionary

In [14]:
s2017_results = createSeasonDict(2017)
df_2017_results = pd.DataFrame(s2016_results)

In [7]:
df_2017_results[1211] #Stats for Gonzaga, for example

avg3s_made                             8.78125
avg_assists                           15.65625
avg_points_allowed                    66.75000
avg_points_scored                     79.50000
avg_rebounds                          44.56250
avg_steals                             5.75000
avg_turnovers                         12.96875
conference_chap_this_year              0.00000
conference_tourney_champ_this_year     0.00000
num_championships                      0.00000
num_tourney_appearances               20.00000
num_wins                              25.00000
sos                                    4.23000
srs                                   16.92000
tournament_seed                       11.00000
Name: 1211, dtype: float64

In [8]:
tourney_detail.columns

Index(['Season', 'Daynum', 'Wteam', 'Wscore', 'Lteam', 'Lscore', 'Wloc',
       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr',
       'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
       'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf'],
      dtype='object')

In [9]:
# Headers of winning and losing team information
winHeaders = ['Wteam', 'Wscore', 'Wfgm', 'Wfga', 
              'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 
             'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf']

lossHeaders = ['Lteam', 'Lscore', 'Lfgm', 'Lfga', 
               'Lfgm3', 'Lfga3','Lftm', 'Lfta', 'Lor', 
               'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf']

In [11]:
# Initialize the new dataframe
df =  pd.DataFrame(data=None, index=range(0, tourney_detail.shape[0]), 
            columns=['Ateam', 'Ascore', 'Afgm', 'Afga', 
              'Afgm3', 'Afga3', 'Aftm', 'Afta', 'Aor', 
             'Adr', 'Aast', 'Ato', 'Astl', 'Ablk', 'Apf', 'Bteam', 'Bscore', 'Bfgm', 'Bfga', 
              'Bfgm3', 'Bfga3', 'Bftm', 'Bfta', 'Bor', 
             'Bdr', 'Bast', 'Bto', 'Bstl', 'Bblk', 'Bpf', 'Awin'], dtype=None, copy=False)

#Initialize Headers
aHeaders = ['Ateam', 'Ascore', 'Afgm', 'Afga', 'Afgm3', 
            'Afga3', 'Aftm', 'Afta', 'Aor', 'Adr', 'Aast', 
            'Ato', 'Astl', 'Ablk', 'Apf']

bHeaders = ['Bteam', 'Bscore', 'Bfgm', 'Bfga', 
              'Bfgm3', 'Bfga3', 'Bftm', 'Bfta', 'Bor', 
             'Bdr', 'Bast', 'Bto', 'Bstl', 'Bblk', 'Bpf']

In [ ]:
import numpy as np

for i in range(0, tourney_detail.shape[0]):
    # Random Int of 1 or 0 decides whether the winning team is team A or team B
    if (np.random.randint(2)):
        aInsertHeaders = winHeaders
        bInsertHeaders = lossHeaders
        df['Awin'][i] = 1
    else:
        aInsertHeaders = lossHeaders
        bInsertHeaders = winHeaders
        df['Awin'][i] = 0
    
    # Puts the team data in the appropriate location in the df
    for j in range(0, len(aHeaders)):
        df[aHeaders[j]][i] = tourney_detail[aInsertHeaders[j]][i]
        df[bHeaders[j]][i] = tourney_detail[bInsertHeaders[j]][i]



In [ ]:
print(df.head())

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

columns= ['Ateam', 'Ascore', 'Afgm', 'Afga', 
              'Afgm3', 'Afga3', 'Aftm', 'Afta', 'Aor', 
             'Adr', 'Aast', 'Ato', 'Astl', 'Ablk', 'Apf', 'Bteam', 'Bscore', 'Bfgm', 'Bfga', 
              'Bfgm3', 'Bfga3', 'Bftm', 'Bfta', 'Bor', 
             'Bdr', 'Bast', 'Bto', 'Bstl', 'Bblk', 'Bpf']

clf = AdaBoostClassifier(n_estimators=100)

Y = list(df["Awin"].values)
X = df[list(columns)].values
clf = clf.fit(X, Y)
scores = cross_val_score(clf, X, Y)
print(scores.mean())

In [ ]:
for i in range(0, len(columns)):
    print(columns[i], ":", clf.feature_importances_[i])

In [81]:
Submission = pd.DataFrame(SampleSubmission['Id'].str.split("_", 3, expand=True))
Submission.rename(columns={0:"Year", 1:"Team1", 2:"Team2"}, inplace=True)

In [87]:
# For row in Submission
# Randomly choose team1 or team2 to be A, other B
# New row in new_pd with cols TeamA TeamB TeamA... TeamB...
# How do we have a measure of which team won? - like what is the "Y" for this?
# Maybe add season aggregated data to the previous model

season_matches_season_stats_pd =  pd.DataFrame(data=None, 
            columns=['A_avg3s_made', 'A_avg_assists', 'A_avg_points_allowed', 
                     'A_avg_points_scored', 'A_avg_rebounds', 'A_avg_steals', 'A_avg_turnovers', 
                     'A_conference_chap_this_year', 'A_conference_tourney_champ_this_year', 
                     'A_num_championships', 'A_num_tourney_appearances', 'A_num_wins', 'A_sos',
                     'A_srs', 'A_tournament_seed',
                     'B_avg3s_made', 'B_avg_assists', 'B_avg_points_allowed', 
                     'B_avg_points_scored', 'B_avg_rebounds', 'B_avg_steals', 'B_avg_turnovers', 
                     'B_conference_chap_this_year', 'B_conference_tourney_champ_this_year', 
                     'B_num_championships', 'B_num_tourney_appearances', 'B_num_wins', 'B_sos',
                     'B_srs', 'B_tournament_seed',
                     'A_win'], dtype=None, copy=False)

season_stats_headers = ['avg3s_made', 'avg_assists', 'avg_points_allowed', 'avg_points_scored',
                        'avg_rebounds', 'avg_steals', 'avg_turnovers',
                        'conference_chap_this_year', 'conference_tourney_champ_this_year',
                        'num_championships', 'num_tourney_appearances', 'num_wins', 'sos',
                        'srs', 'tournament_seed']

a_season_Headers = ['A_avg3s_made', 'A_avg_assists', 'A_avg_points_allowed', 
                     'A_avg_points_scored', 'A_avg_rebounds', 'A_avg_steals', 'A_avg_turnovers', 
                     'A_conference_chap_this_year', 'A_conference_tourney_champ_this_year', 
                     'A_num_championships', 'A_num_tourney_appearances', 'A_num_wins', 'A_sos',
                     'A_srs', 'A_tournament_seed']

b_season_Headers = ['B_avg3s_made', 'B_avg_assists', 'B_avg_points_allowed', 
                     'B_avg_points_scored', 'B_avg_rebounds', 'B_avg_steals', 'B_avg_turnovers', 
                     'B_conference_chap_this_year', 'B_conference_tourney_champ_this_year', 
                     'B_num_championships', 'B_num_tourney_appearances', 'B_num_wins', 'B_sos',
                     'B_srs', 'B_tournament_seed']

a_headers_mapping = dict(zip(season_stats_headers, a_season_Headers))
b_headers_mapping = dict(zip(season_stats_headers, b_season_Headers))

reg_season_game_outcomes = pd.DataFrame.copy(reg_season_detail)
reg_season_game_outcomes = reg_season_game_outcomes.drop(['Wscore', 'Daynum','Lscore','Wloc',
                                                       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 
                                                        'Wfta', 'Wor', 'Wdr','Wast', 'Wto', 'Wstl', 'Wblk',
                                                        'Wpf', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 
                                                        'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf'], axis=1)
reg_season_game_outcomes = reg_season_game_outcomes[reg_season_game_outcomes.Season == 2017]
reg_season_game_outcomes = reg_season_game_outcomes.drop("Season", axis=1)

In [88]:
import numpy as np

print(reg_season_game_outcomes.tail())
#71241, 76635
#print(b_headers_mapping)

df_2017_results[1211]
w_team_season_stats = df_2017_results[1211]
#print(w_team_season_stats.index)
 
all_headers = list(a_headers_mapping.values()) + list(b_headers_mapping.values()) + ["A_win_flag"]
#print(all_headers)
season_training = pd.DataFrame(columns=all_headers)

#print(reg_season_game_outcomes)    


for index, row in reg_season_game_outcomes.iterrows():
    w_team_season_stats = df_2017_results[row["Wteam"]]
    l_team_season_stats = df_2017_results[row["Lteam"]]
    
    #print(w_team_season_stats)
    
    if(np.random.randint(2)):
        # Winning Team :: A
        A_win_flag = 1
        A_team_season_stats = w_team_season_stats.rename(a_headers_mapping)
        #A_team_season_stats.set_value("A_team", row["Wteam"])
        B_team_season_stats = l_team_season_stats.rename(b_headers_mapping)
        #B_team_season_stats.set_value("B_team", row["Lteam"])

    else:
        # Winning Team :: B
        A_win_flag = 0
        A_team_season_stats = l_team_season_stats.rename(a_headers_mapping)
        #A_team_season_stats.set_value("A_team", row["Lteam"])
        B_team_season_stats = w_team_season_stats.rename(b_headers_mapping)
        #B_team_season_stats.set_value("B_team", row["Wteam"])

        
    matchup = A_team_season_stats.append(B_team_season_stats)
    matchup = matchup.set_value("A_win", A_win_flag)
    season_matches_season_stats_pd.loc[index]= matchup

print(season_matches_season_stats_pd)
    
    #print(l_team_season_stats)
    # print(df_2016_results[row["Wteam"]]
    # a_team, b_team, season_stats for a_team, season_stats b_team, a_win
    # a_team from reg_season_game_outcomes
    
#TODO:
#randomly make Wteam A or B
#create A_win var appropriately
# new row in df comprised of TeamA Team B TeamA_stats TeamB_stats A_win

       Wteam  Lteam
76631   1276   1458
76632   1343   1463
76633   1348   1433
76634   1374   1153
76635   1407   1402
       A_avg3s_made  A_avg_assists  A_avg_points_allowed  A_avg_points_scored  \
71241      9.384615      15.230769             68.384615            71.115385   
71242      6.064516      13.225806             65.709677            74.612903   
71243      9.441176      21.147059             63.382353            79.764706   
71244      7.937500      12.656250             73.968750            73.281250   
71245      6.750000      16.187500             74.687500            78.156250   
71246      8.774194      11.354839             78.096774            70.709677   
71247      9.838710      16.516129             66.516129            69.709677   
71248      6.562500      18.843750             69.625000            76.312500   
71249      8.629630      11.185185             79.481481            70.962963   
71250      7.483871      14.064516             73.161290            68

In [91]:
## Create Testing Data
Submission["Team1"]

tourney_matchups_season_stats_pd =  pd.DataFrame(data=None, 
            columns=['A_avg3s_made', 'A_avg_assists', 'A_avg_points_allowed', 
                     'A_avg_points_scored', 'A_avg_rebounds', 'A_avg_steals', 'A_avg_turnovers', 
                     'A_conference_chap_this_year', 'A_conference_tourney_champ_this_year', 
                     'A_num_championships', 'A_num_tourney_appearances', 'A_num_wins', 'A_sos',
                     'A_srs', 'A_tournament_seed',
                     'B_avg3s_made', 'B_avg_assists', 'B_avg_points_allowed', 
                     'B_avg_points_scored', 'B_avg_rebounds', 'B_avg_steals', 'B_avg_turnovers', 
                     'B_conference_chap_this_year', 'B_conference_tourney_champ_this_year', 
                     'B_num_championships', 'B_num_tourney_appearances', 'B_num_wins', 'B_sos',
                     'B_srs', 'B_tournament_seed',
                     'A_win'], dtype=None, copy=False)

for index, row in Submission.iterrows():
    team_season_stats_1 = df_2017_results[int(row["Team1"])]
    team_season_stats_2 = df_2017_results[int(row["Team2"])]
    
    if(np.random.randint(2)):
        # Team1 :: A
        A_team_season_stats = team_season_stats_1.rename(a_headers_mapping)
        B_team_season_stats = team_season_stats_2.rename(b_headers_mapping)

    else:
        # Team1 :: B
        A_team_season_stats = team_season_stats_2.rename(a_headers_mapping)
        B_team_season_stats = team_season_stats_1.rename(b_headers_mapping)

        
    matchup = A_team_season_stats.append(B_team_season_stats)
    tourney_matchups_season_stats_pd.loc[index]= matchup

print(tourney_matchups_season_stats_pd)

      A_avg3s_made  A_avg_assists  A_avg_points_allowed  A_avg_points_scored  \
0         6.696970      14.696970             69.000000            81.212121   
1         6.562500      18.843750             69.625000            76.312500   
2         6.696970      14.696970             69.000000            81.212121   
3         6.696970      14.696970             69.000000            81.212121   
4         6.696970      14.696970             69.000000            81.212121   
5         6.696970      14.696970             69.000000            81.212121   
6         6.906250      15.000000             65.812500            73.187500   
7         9.939394      14.484848             72.060606            81.454545   
8         9.218750      13.968750             74.812500            76.218750   
9         6.413793      16.655172             72.862069            75.586207   
10        6.696970      14.696970             69.000000            81.212121   
11        6.696970      14.696970       

In [115]:
# Train AdaBoost Classifier on Training Data

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
bagging = BaggingClassifier(KNeighborsClassifier(), n_estimators=100, bootstrap_features=True,
                            max_samples=0.5, max_features=0.5)

columns= ['A_avg3s_made', 'A_avg_assists', 'A_avg_points_allowed', 
                     'A_avg_points_scored', 'A_avg_rebounds', 'A_avg_steals', 'A_avg_turnovers', 
                     'A_conference_chap_this_year', 'A_conference_tourney_champ_this_year', 
                     'A_num_championships', 'A_num_tourney_appearances', 'A_num_wins', 'A_sos',
                     'A_srs', 'A_tournament_seed',
                     'B_avg3s_made', 'B_avg_assists', 'B_avg_points_allowed', 
                     'B_avg_points_scored', 'B_avg_rebounds', 'B_avg_steals', 'B_avg_turnovers', 
                     'B_conference_chap_this_year', 'B_conference_tourney_champ_this_year', 
                     'B_num_championships', 'B_num_tourney_appearances', 'B_num_wins', 'B_sos',
                     'B_srs', 'B_tournament_seed']

#clf = RandomForestClassifier(n_estimators=1000, oob_score = True, max_features = None)

Y = list(season_matches_season_stats_pd["A_win"].values)
X = season_matches_season_stats_pd[list(columns)].values
#clf = clf.fit(X, Y)
bagging = bagging.fit(X, Y)
scores = cross_val_score(bagging, X, Y)
print(scores.mean())

0.667276530466


In [116]:
# Test 

testallX = tourney_matchups_season_stats_pd[list(columns)].values
probabilities = clf.predict_proba(testallX)


for i in range(0,len(probabilities)):
    print(probabilities[i][1])
    
#print(clf.predict(testallX))

0.6944
0.24905952381
0.962333333333
0.209933333333
0.357488888889
0.661866666667
0.17445
0.683793650794
0.0481166666667
0.091
0.587897619048
0.796055555556
0.562466666667
0.926738888889
0.7858
0.9645
0.666266666667
0.250125
0.946666666667
0.590445238095
0.599554761905
0.205689285714
0.810983333333
0.294321428571
0.607771428571
0.198557142857
0.824433333333
0.930916666667
0.8685
0.0557833333333
0.0395
0.11475
0.170333333333
0.064
0.442833333333
0.95625
0.8549
0.3825
0.141602380952
0.349561904762
0.134733333333
0.318933333333
0.303511904762
0.1803
0.948666666667
0.611722222222
0.316269047619
0.150133333333
0.751925
0.0521666666667
0.972
0.019
0.507332936508
0.085
0.2048
0.302188095238
0.404388095238
0.959
0.437430952381
0.628128571429
0.797533333333
0.240733333333
0.325536904762
0.38025
0.0737833333333
0.808466666667
0.209033333333
0.494794047619
0.145566666667
0.589301587302
0.55205952381
0.66679484127
0.609426190476
0.663324603175
0.0992916666667
0.193683333333
0.938833333333
0.5681142

In [117]:
import csv
all_matchid = []

for index, row in Submission.iterrows():
    matchid = row["Year"] +"_" + row["Team1"] + "_" + row["Team2"]
    all_matchid.append(matchid)

with open("submission2017.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'pred'])
    for idx, val in enumerate(all_matchid):
        if False:
            writer.writerow([val, .5])
        else:
            writer.writerow([val, probabilities[idx][1]])
        #replace .5 with scores

['2017_1112_1116', '2017_1112_1124', '2017_1112_1137', '2017_1112_1139', '2017_1112_1153', '2017_1112_1166', '2017_1112_1173', '2017_1112_1181', '2017_1112_1190', '2017_1112_1195', '2017_1112_1196', '2017_1112_1199', '2017_1112_1211', '2017_1112_1233', '2017_1112_1235', '2017_1112_1240', '2017_1112_1242', '2017_1112_1243', '2017_1112_1245', '2017_1112_1246', '2017_1112_1257', '2017_1112_1266', '2017_1112_1268', '2017_1112_1274', '2017_1112_1276', '2017_1112_1277', '2017_1112_1278', '2017_1112_1291', '2017_1112_1292', '2017_1112_1297', '2017_1112_1300', '2017_1112_1305', '2017_1112_1308', '2017_1112_1309', '2017_1112_1314', '2017_1112_1315', '2017_1112_1321', '2017_1112_1323', '2017_1112_1329', '2017_1112_1332', '2017_1112_1343', '2017_1112_1344', '2017_1112_1345', '2017_1112_1348', '2017_1112_1355', '2017_1112_1371', '2017_1112_1374', '2017_1112_1376', '2017_1112_1388', '2017_1112_1407', '2017_1112_1411', '2017_1112_1413', '2017_1112_1417', '2017_1112_1423', '2017_1112_1425', '2017_111